In [1]:

import os
import numpy as np
import matplotlib.pyplot as plt
split = "test"
dest_path = "/liuzicheng/ljh/hyena-dna/data/splicing_prediction"
all_seqs = np.load(os.path.join(dest_path,  split + "_seq.npy"), allow_pickle=True)
all_labels = np.load(os.path.join(dest_path,  split + "_target.npy"), allow_pickle=True)


In [9]:
hyena_path='/liuzicheng/ljh/hyena-dna/outputs/2024-04-03/15-12-52-724312/checkpoints/val/pr_auc_mean.ckpt'
mamba_path='/liuzicheng/ljh/hyena-dna/outputs/2024-04-20/02-29-01-651292/checkpoints/val/pr_auc_mean.ckpt'
NT_path='/liuzicheng/ljh/hyena-dna/outputs/2024-04-08/01-41-26-952782/checkpoints/val/pr_auc_mean.ckpt'
genalm_path='/liuzicheng/ljh/hyena-dna/outputs/2024-04-06/11-21-47-532494/checkpoints/val/pr_auc_mean.ckpt'
bert2_path='/liuzicheng/ljh/hyena-dna/outputs/2024-04-02/18-41-34-140615/checkpoints/val/pr_auc_mean.ckpt'
batch_size=1
max_length=3000

import torch
from sklearn.metrics import f1_score, roc_auc_score
def pr_auc(logits,y):
    # compute metrics based on stored labels, predictions, ...
    metrics = {}
    y, p = y, torch.sigmoid(logits)
    #convert three dimension into 2 dimension
    y=y.reshape(-1,y.shape[-1]).cpu().numpy()
    p=p.reshape(-1,p.shape[-1]).cpu().detach().numpy()
    # compute pr-auc for each class independetly
    for label in [0, 1, 2]:
        y_label = y[:, label]
        p_label = p[:, label]
        if not np.isnan(p_label).any():
            try:
                pr_auc = roc_auc_score(y_label, p_label)
            except ValueError:
                #calculate accurate rate for this label
                #convert p_label to 0-1
                p_label = (p_label > 0.5).astype(int)
                accurate_rate = (y_label == p_label).sum() / len(y_label)
                #if accurate rate is 1, set pr_auc to 1
                pr_auc = accurate_rate
        else:
            pr_auc = np.nan
        # to be compatible with sklearn 1.1+
        metrics[f'pr_auc_{label}'] = pr_auc if not np.isnan(pr_auc) else 0.0
    metrics['pr_auc_mean'] = (metrics['pr_auc_1'] + metrics['pr_auc_2']+metrics['pr_auc_0']) / 3
    return metrics

def accuracy(logits, y):
    logits = logits.view(-1, logits.shape[-1])
    preds = torch.argmax(logits, dim=-1)
    if y.numel() > logits.shape[0]:
        # Mixup leads to this case: use argmax class
        y = y.argmax(dim=-1)
    y = y.view(-1)
    return torch.eq(preds, y).float().mean()

In [10]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn


with torch.no_grad():
    state_dict='/liuzicheng/ljh/hyena-dna/weight/hyenadna/hyenadna-large-1m-seqlen'
    hyena_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    hyena_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load(hyena_path)['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    hyena_decoder = nn.Linear(256,3).to('cuda')
    #edit key name in hyena_decoder
    
    hyena_model.load_state_dict(checkpoint,strict=False)
    hyena_decoder.load_state_dict(checkpoint,strict=False)
    hyena_model.eval()
    hyena_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):
        sequence_encoded=hyena_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i][:1000]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=hyena_model(input_ids=seqs).last_hidden_state
        hidden_states=hidden_states[..., :1000, :]
        out1=hyena_decoder(hidden_states)
        out1_hyena=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_hyena)
        seq_list_numpy=np.array(seq_list)

        target_list_numpy=np.array(target_list)
        seq_list_tensor=torch.FloatTensor(seq_list_numpy)
        target_list_tensor=torch.FloatTensor(target_list_numpy)
        #calculate the 
        if i>=1:
            
            
            if i%1000==0:
                print(i)
                pearsonr=pr_auc(seq_list_tensor,target_list_tensor)
                print(pearsonr)
                accuracy_value=accuracy(seq_list_tensor,target_list_tensor)
                print(accuracy_value)
            

#plot the bar plot of the pearsonr


1000
{'pr_auc_0': 0.5154864462490805, 'pr_auc_1': 0.5590537978461373, 'pr_auc_2': 0.6995160261908133, 'pr_auc_mean': 0.5913520900953437}
tensor(0.9996)
2000
{'pr_auc_0': 0.5141785628747122, 'pr_auc_1': 0.5923108638222884, 'pr_auc_2': 0.7170157376860257, 'pr_auc_mean': 0.6078350547943421}
tensor(0.9996)
3000
{'pr_auc_0': 0.5525250050826079, 'pr_auc_1': 0.5910793756973244, 'pr_auc_2': 0.7207239376237361, 'pr_auc_mean': 0.6214427728012227}
tensor(0.9996)


KeyboardInterrupt: 

In [8]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn


max_length=3000
with torch.no_grad():
    state_dict='/liuzicheng/ljh/hyena-dna/weight/dnabert2'
    bert2_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    bert2_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load(bert2_path)['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    bert2_decoder = nn.Linear(768,3).to('cuda')
    #edit key name in hyena_decoder
    
    bert2_model.load_state_dict(checkpoint,strict=False)
    bert2_decoder.load_state_dict(checkpoint,strict=False)
    bert2_model.eval()
    bert2_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=bert2_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i][:1000]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=bert2_model(input_ids=seqs,export_hidden_states=True)[0]
        hidden_states=hidden_states[..., :1000, :]
        out1=bert2_decoder(hidden_states)
        out1_bert2=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_bert2)
        seq_list_numpy=np.array(seq_list)

        target_list_numpy=np.array(target_list)
        seq_list_tensor=torch.FloatTensor(seq_list_numpy)
        target_list_tensor=torch.FloatTensor(target_list_numpy)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
                pearsonr=pr_auc(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


/root/.cache/huggingface/modules/transformers_modules/dnabert2/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at /liuzicheng/ljh/hyena-dna/weight/dnabert2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/.cache/huggingface/modules/transformers_modules/dnabert2/bert_layers.py:433: UserWarning: Increasing alibi size from 512 to 3000
  warnings.warn(


1000
{'pr_auc_0': 0.4413639488404158, 'pr_auc_1': 0.6495801693186615, 'pr_auc_2': 0.7035993527238612, 'pr_auc_mean': 0.5981811569609795}
2000
{'pr_auc_0': 0.4833810037010731, 'pr_auc_1': 0.6579949162945948, 'pr_auc_2': 0.7102444025825281, 'pr_auc_mean': 0.617206774192732}
3000
{'pr_auc_0': 0.5768825078913379, 'pr_auc_1': 0.6441416139443331, 'pr_auc_2': 0.7067245205190908, 'pr_auc_mean': 0.6425828807849205}
4000
{'pr_auc_0': 0.6094680668758002, 'pr_auc_1': 0.6325235272635453, 'pr_auc_2': 0.7039271875574811, 'pr_auc_mean': 0.6486395938989422}
5000
{'pr_auc_0': 0.6276391536145002, 'pr_auc_1': 0.6317029275923831, 'pr_auc_2': 0.7048822524375558, 'pr_auc_mean': 0.6547414445481463}
6000
{'pr_auc_0': 0.6320016902614787, 'pr_auc_1': 0.6241559409749575, 'pr_auc_2': 0.6982471967887951, 'pr_auc_mean': 0.6514682760084104}
7000
{'pr_auc_0': 0.6455787283720203, 'pr_auc_1': 0.6287807207037904, 'pr_auc_2': 0.7012021169564384, 'pr_auc_mean': 0.6585205220107498}
8000
{'pr_auc_0': 0.6472556370414385, 'pr_

In [10]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn


max_length=3000
with torch.no_grad():
    state_dict='/liuzicheng/ljh/hyena-dna/weight/genalm/gena-lm-bigbird-base-t2t'
    genalm_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    genalm_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load(genalm_path)['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    genalm_decoder = nn.Linear(768,3).to('cuda')
    #edit key name in hyena_decoder
    
    genalm_model.load_state_dict(checkpoint,strict=False)
    genalm_decoder.load_state_dict(checkpoint,strict=False)
    genalm_model.eval()
    genalm_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=genalm_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i][:1000]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=genalm_model(input_ids=seqs, output_hidden_states=True,).hidden_states[-1]
        hidden_states=hidden_states[..., :1000, :]
        out1=genalm_decoder(hidden_states)
        out1_genalm=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_genalm)
        seq_list_numpy=np.array(seq_list)

        target_list_numpy=np.array(target_list)
        seq_list_tensor=torch.FloatTensor(seq_list_numpy)
        target_list_tensor=torch.FloatTensor(target_list_numpy)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
                pearsonr=pr_auc(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


1000
{'pr_auc_0': 0.7880212471259458, 'pr_auc_1': 0.6770300582523869, 'pr_auc_2': 0.7187810802183864, 'pr_auc_mean': 0.7279441285322396}
2000
{'pr_auc_0': 0.8167638615468158, 'pr_auc_1': 0.6561324144945506, 'pr_auc_2': 0.7148380653496311, 'pr_auc_mean': 0.7292447804636658}
3000
{'pr_auc_0': 0.8478734619509036, 'pr_auc_1': 0.6513071565600782, 'pr_auc_2': 0.7241171135823166, 'pr_auc_mean': 0.7410992440310995}
4000
{'pr_auc_0': 0.851880441791636, 'pr_auc_1': 0.6420799196727746, 'pr_auc_2': 0.7283626722476464, 'pr_auc_mean': 0.7407743445706857}
5000
{'pr_auc_0': 0.8597831673439507, 'pr_auc_1': 0.6354504501119524, 'pr_auc_2': 0.7304985274249985, 'pr_auc_mean': 0.7419107149603006}
6000
{'pr_auc_0': 0.8677362138412498, 'pr_auc_1': 0.6283971543377364, 'pr_auc_2': 0.7241814758055377, 'pr_auc_mean': 0.7401049479948413}
7000
{'pr_auc_0': 0.8611445707305545, 'pr_auc_1': 0.6311706598652306, 'pr_auc_2': 0.7273982069457894, 'pr_auc_mean': 0.7399044791805247}
8000
{'pr_auc_0': 0.8626132615327134, 'pr_

In [3]:
from transformers import AutoTokenizer,AutoModel,AutoModelForMaskedLM
import torch
from torch import nn


max_length=3000
with torch.no_grad():
    state_dict='/liuzicheng/ljh/hyena-dna/weight/nt/nucleotide-transformer-v2-500m-multi-species'
    nt_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    nt_model=AutoModelForMaskedLM.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load(NT_path)['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    nt_decoder = nn.Linear(1024,3).to('cuda')
    #edit key name in hyena_decoder
    
    nt_model.load_state_dict(checkpoint,strict=False)
    nt_decoder.load_state_dict(checkpoint,strict=False)
    nt_model.eval()
    nt_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=nt_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i][:1000]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=nt_model(input_ids=seqs,output_hidden_states=True)['hidden_states'][-1]
        hidden_states=hidden_states[..., :1000, :]
        out1=nt_decoder(hidden_states)
        out1_nt=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_nt)
        seq_list_numpy=np.array(seq_list)

        target_list_numpy=np.array(target_list)
        seq_list_tensor=torch.FloatTensor(seq_list_numpy)
        target_list_tensor=torch.FloatTensor(target_list_numpy)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
                pearsonr=pr_auc(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


/liuzicheng/anaconda3/envs/evo-design/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1000
{'pr_auc_0': 0.5528131862270298, 'pr_auc_1': 0.5463983841379774, 'pr_auc_2': 0.7100390064366782, 'pr_auc_mean': 0.6030835256005618}
2000
{'pr_auc_0': 0.5644413401418739, 'pr_auc_1': 0.5677631092582998, 'pr_auc_2': 0.7177888964563967, 'pr_auc_mean': 0.6166644486188567}
3000
{'pr_auc_0': 0.579306468454271, 'pr_auc_1': 0.5640886874766704, 'pr_auc_2': 0.7190675758194758, 'pr_auc_mean': 0.6208209105834724}
4000
{'pr_auc_0': 0.6037103479946816, 'pr_auc_1': 0.5592286949901967, 'pr_auc_2': 0.7166983058490674, 'pr_auc_mean': 0.6265457829446485}
5000
{'pr_auc_0': 0.6150882743431582, 'pr_auc_1': 0.5549603588757929, 'pr_auc_2': 0.7193207428935381, 'pr_auc_mean': 0.6297897920374963}
6000
{'pr_auc_0': 0.6189362088925453, 'pr_auc_1': 0.5541015259108296, 'pr_auc_2': 0.7131134184647558, 'pr_auc_mean': 0.6287170510893768}
7000
{'pr_auc_0': 0.6063457937701472, 'pr_auc_1': 0.5553964152469485, 'pr_auc_2': 0.7159778537577799, 'pr_auc_mean': 0.6259066875916252}
8000
{'pr_auc_0': 0.5966400669665272, 'pr_

In [4]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn


max_length=3000
with torch.no_grad():
    state_dict='/liuzicheng/ljh/hyena-dna/weight/mamba/caduceus-ph_seqlen-131k_d_model-256_n_layer-16'
    mamba_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    mamba_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load(mamba_path)['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    mamba_decoder = nn.Linear(256,3).to('cuda')
    #edit key name in hyena_decoder
    
    mamba_model.load_state_dict(checkpoint,strict=False)
    mamba_decoder.load_state_dict(checkpoint,strict=False)
    mamba_model.eval()
    mamba_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=mamba_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i][:1000]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=mamba_model(seqs,output_hidden_states=True).last_hidden_state
        hidden_states=hidden_states[..., :1000, :]
        out1=mamba_decoder(hidden_states)
        out1_mamba=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_mamba)
        seq_list_numpy=np.array(seq_list)

        target_list_numpy=np.array(target_list)
        seq_list_tensor=torch.FloatTensor(seq_list_numpy)
        target_list_tensor=torch.FloatTensor(target_list_numpy)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
                pearsonr=pr_auc(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


1000
{'pr_auc_0': 0.5388617909092803, 'pr_auc_1': 0.6441216147249552, 'pr_auc_2': 0.7270903694081531, 'pr_auc_mean': 0.6366912583474629}
2000
{'pr_auc_0': 0.5433083804365973, 'pr_auc_1': 0.6490626569738048, 'pr_auc_2': 0.7270350315210441, 'pr_auc_mean': 0.6398020229771487}
3000
{'pr_auc_0': 0.6076680475028369, 'pr_auc_1': 0.6532792935822576, 'pr_auc_2': 0.7362385209274198, 'pr_auc_mean': 0.665728620670838}
4000
{'pr_auc_0': 0.6358498053252069, 'pr_auc_1': 0.6416271479971765, 'pr_auc_2': 0.7351133119952585, 'pr_auc_mean': 0.6708634217725473}
5000
{'pr_auc_0': 0.6456392775320324, 'pr_auc_1': 0.6457690367912082, 'pr_auc_2': 0.74072134952049, 'pr_auc_mean': 0.6773765546145768}
6000
{'pr_auc_0': 0.6539994429578858, 'pr_auc_1': 0.6367329557369094, 'pr_auc_2': 0.7296913051219482, 'pr_auc_mean': 0.6734745679389144}
7000
{'pr_auc_0': 0.6613370710411016, 'pr_auc_1': 0.637248168292053, 'pr_auc_2': 0.7323301995997362, 'pr_auc_mean': 0.6769718129776302}
8000
{'pr_auc_0': 0.6480404693737357, 'pr_auc

In [22]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn
import sys
sys.path.append('/liuzicheng/ljh/hyena-dna/')
from src.models.sequence.SpliceAI import SpliceAI

def genomic_to_one_hot(genomic_sequence):
        mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
        one_hot = np.zeros((len(genomic_sequence), 4))
        for i, base in enumerate(genomic_sequence):
            if base in mapping:
                one_hot[i, mapping[base]] = 1
            else:
                # 如果碱基不是A、C、G、T或N，可以选择将其编码为全零向量或者平均分配概率
                one_hot[i, :] = 0.25  # 或者使用 np.full((5,), 0.2) 平均分配概率
        return one_hot
max_length=3000
with torch.no_grad():
    splice=SpliceAI(3000,1000)
    full_sequence=[]
    path='/liuzicheng/ljh/hyena-dna/outputs/2024-05-13/05-05-19-201408/checkpoints/val/pr_auc_mean.ckpt'
    checkpoint=torch.load(path)['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    

    #edit key name in hyena_decoder
    
    splice.load_state_dict(checkpoint,strict=False)
    splice.to('cuda')
    splice.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        seq=genomic_to_one_hot(all_seqs[i])
        seq_ids = torch.from_numpy(seq).float()[:max_length,:]
        target = all_labels[i][:1000]
        
        seqs=torch.reshape(seq_ids,(1,max_length,-1)).to('cuda')
        target_list.append(target)
        hidden_states=splice(seqs)
        out1_splice=hidden_states.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_splice)
        seq_list_numpy=np.array(seq_list)

        target_list_numpy=np.array(target_list)
        seq_list_tensor=torch.FloatTensor(seq_list_numpy)
        target_list_tensor=torch.FloatTensor(target_list_numpy)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
                pearsonr=pr_auc(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


1000
{'pr_auc_0': 0.5012496497312322, 'pr_auc_1': 0.557875898625051, 'pr_auc_2': 0.6543104627247216, 'pr_auc_mean': 0.5711453370270017}
2000
{'pr_auc_0': 0.513053010096482, 'pr_auc_1': 0.5442053543889976, 'pr_auc_2': 0.6700020347770071, 'pr_auc_mean': 0.5757534664208289}
3000
{'pr_auc_0': 0.5559023029434687, 'pr_auc_1': 0.5489443397389752, 'pr_auc_2': 0.6737774871791081, 'pr_auc_mean': 0.5928747099538506}
4000
{'pr_auc_0': 0.5839581145781151, 'pr_auc_1': 0.5398353736083129, 'pr_auc_2': 0.6743729276059067, 'pr_auc_mean': 0.5993888052641115}
5000
{'pr_auc_0': 0.5937846155051626, 'pr_auc_1': 0.5435915583256301, 'pr_auc_2': 0.6789547593555947, 'pr_auc_mean': 0.6054436443954625}
6000
{'pr_auc_0': 0.583674049685009, 'pr_auc_1': 0.5411016133705435, 'pr_auc_2': 0.6767005084553682, 'pr_auc_mean': 0.600492057170307}
7000
{'pr_auc_0': 0.5889916726101043, 'pr_auc_1': 0.5396621572715593, 'pr_auc_2': 0.6793858598361087, 'pr_auc_mean': 0.6026798965725908}
8000
{'pr_auc_0': 0.5768482524809757, 'pr_auc